# FeatureEngineering

## Summary
- 문제 : dataset의 큰 용량 때문에 컴퓨터가 연속된 연산을 못하고 계속 퍼지는 현상이 발생.
- 해결 : Feature Engineering을 독립적으로 수행해서 그 결과값을 .csv 파일로 만들어 놓음.
- 이를 통해 연산을 분리해서 컴퓨터의 부담을 경감시킴.
- 앞서의 holidays_events, oil dataset에 대한 feature engineering 의 결과를 적용하고,
- train / oil / items / holidays_events / stores dataset들을 merge하여 modeling에 바로 사용할 수 있는 dataset 생성

In [1]:
%%time
import pandas as pd
import numpy as np
import csv

import datetime
from datetime import date, timedelta

dtypes = {'store_nbr': np.dtype('int8'),
          'id': np.dtype('int32'),
          'item_nbr': np.dtype('int32'),
          'unit_sales': np.dtype('float64')}

df_train = pd.read_csv('./data/train_drop_pro.csv', dtype=dtypes)
df_test = pd.read_csv('./data/test.csv', dtype=dtypes)
df_oil = pd.read_csv('./data/oil.csv')
df_items = pd.read_csv('./data/items.csv')
df_holidays = pd.read_csv('./data/holidays_events.csv')
df_stores = pd.read_csv('./data/stores.csv')


CPU times: user 1min 5s, sys: 6.35 s, total: 1min 11s
Wall time: 1min 12s


In [2]:
%%time
# Replace missing_values in oil_data
# reference : https://www.kaggle.com/kaggleslayer/grocery-prediction-with-neural-network
min_oil_date = min(df_train.date)
max_oil_date = max(df_test.date)

calendar = []

d1 = datetime.datetime.strptime(min_oil_date, '%Y-%m-%d')
d2 = datetime.datetime.strptime(max_oil_date, '%Y-%m-%d')

delta = d2 - d1

for i in range(delta.days + 1):
    calendar.append(datetime.date.strftime(d1 + timedelta(days=i), '%Y-%m-%d'))
    
calendar = pd.DataFrame({'date':calendar})

df_oil = calendar.merge(df_oil, left_on='date', right_on='date', how='left')

na_index_oil = df_oil[df_oil['dcoilwtico'].isnull() == True].index.values
na_index_oil_plus = na_index_oil.copy()
na_index_oil_minus = np.maximum(0, na_index_oil - 1)

for i in range(len(na_index_oil)):
    k = 1
    while (na_index_oil[min(i+k, len(na_index_oil)-1)] == na_index_oil[i]+k):
        k += 1
    na_index_oil_plus[i] = min(len(df_oil)-1, na_index_oil_plus[i] + k)
    
for i in range(len(na_index_oil)):
    if (na_index_oil[i] == 0):
        df_oil.loc[na_index_oil[i], 'dcoilwtico'] = df_oil.loc[na_index_oil_plus[i], 'dcoilwtico']

    elif (na_index_oil[i] == len(df_oil)):
        df_oil.loc[na_index_oil[i], 'dcoilwtico'] = df_oil.loc[na_index_oil_minus[i], 'dcoilwtico']
    
    else:
        df_oil.loc[na_index_oil[i], 'dcoilwtico'] = (df_oil.loc[na_index_oil_plus[i], 'dcoilwtico'] + df_oil.loc[na_index_oil_minus[i], 
                                                                       'dcoilwtico'])/2

# About earthquake    
df_oil['earthquake'] = 0
df_oil['earthquake'].loc[746] = 1


CPU times: user 2.68 s, sys: 25.3 ms, total: 2.7 s
Wall time: 2.7 s


/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
%%time
# convert holidays_event
df_holidays_sort=df_holidays.sort_values(by='locale')

df_holidays_L = df_holidays_sort.loc[:66]
df_holidays_N = df_holidays_sort.loc[227:154]
df_holidays_R = df_holidays_sort.loc[334:278]
# dropping datasets which was duplicated on 'date'
df_holidays_N = df_holidays_N.drop([35, 40, 156, 235, 242, 245])

# description 의 뒷부분 제거
def desc_transfer(data_set):
    desc_use = data_set['description'].str.split(' ').str[0]
    df_base = data_set.drop('description', axis=1)
    df_desc_transfered = pd.concat([df_base, desc_use], axis=1)
    return df_desc_transfered

df_holidays_L_use = desc_transfer(df_holidays_L)
df_holidays_N_use = desc_transfer(df_holidays_N)
df_holidays_R_use = desc_transfer(df_holidays_R)

# holidays 에서 사용할 데이터만 정리
df_h_n = df_holidays_N_use[['date', 'type', 'description']]
df_h_r = df_holidays_R_use[['date', 'locale_name', 'description']]
df_h_l = df_holidays_L_use[['date', 'locale_name', 'description']]

# national의 description에서 장기연휴의 경우 뒤에 붙어 있는 +/- 를 제거하기
drop_add_plus = df_h_n['description'].str.split('+').str[0]
drop_add_base_p = df_h_n.drop('description', axis=1)
df_h_n_drop_plus = pd.concat([drop_add_base_p, drop_add_plus], axis=1)

drop_add_minus = df_h_n_drop_plus['description'].str.split('-').str[0]
drop_add_base_m = df_h_n_drop_plus.drop('description', axis=1)
df_h_n_drop = pd.concat([drop_add_base_m, drop_add_minus], axis=1)

# locale_name을 regional 과 locale 에 따라 구분하기 편하게 변경함.
df_h_r_rename = df_h_r.rename(index=str, columns={"locale_name": "state"})
df_h_l_rename = df_h_l.rename(index=str, columns={"locale_name": "city"})

# merge 과정에서 df_h_l_rename의 2016-07-24 이 city와 date가 중복됨을 확인함. 둘중 하나를 지움.
df_h_l_rename_drop = df_h_l_rename.drop(['265'])


CPU times: user 12.4 ms, sys: 110 µs, total: 12.5 ms
Wall time: 12.5 ms


In [4]:
# Merge datasets
def merge_df(dataframe):
    df_1 = pd.merge(dataframe, df_items, how='left', on='item_nbr')
    df_2 = pd.merge(df_1, df_stores, how='left', on='store_nbr')
    df_3 = pd.merge(df_2, df_oil, how='left', on='date')
    df_4 = pd.merge(df_3, df_h_n_drop, how='left', on='date', suffixes=('_store', '_holiday'))
    df_5 = pd.merge(df_4, df_h_r_rename, how='left', on=['date', 'state'], suffixes=('', '_r'))
    df_merged = pd.merge(df_5, df_h_l_rename_drop, how='left', on=['date', 'city'], suffixes=('_n', '_l'))

    having_Nan_Columns = ('type_holiday', 'description_n', 'description_r', 'description_l')

    for Column in having_Nan_Columns:
        df_merged[Column] = df_merged[Column].fillna('None')
    
    return df_merged

In [ ]:
# Convert date feature
def convert_date(df):
    df['year'] = df.date.apply(lambda x: x.split('-')[0])
    df['month'] = df.date.apply(lambda x: x.split('-')[1])
    df['day'] = df.date.apply(lambda x: x.split('-')[2])
    df['date'] = pd.to_datetime(df['date'])
    df['week_day'] = df['date'].dt.weekday_name
    df = df.drop('date', axis=1)
    return df

In [5]:
# Execute functions
%%time

df_train = merge_df(df_train)
df_train = convert_date(df_train)

df_test = merge_df(df_test)
df_test = convert_date(df_test)

CPU times: user 10.2 s, sys: 1.98 s, total: 12.2 s
Wall time: 12.2 s


In [6]:
# Make Dataframes to use in Modeling in .csv file
%%time
df_train.to_csv('./data/train_use.csv', index=False)
df_test.to_csv('./data/test_use.csv', index=False)

CPU times: user 29.3 s, sys: 173 ms, total: 29.5 s
Wall time: 29.6 s
